In [2]:
import torch
from huggingface_hub import notebook_login

notebook_login()

In [32]:
from datasets import load_dataset
ds = load_dataset("roneneldan/TinyStories")
ds

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [4]:
from huggingface_hub import create_repo

create_repo("Tiny-Storyteller")

RepoUrl('https://huggingface.co/srinivasan-sridhar28/Tiny-Storyteller', endpoint='https://huggingface.co', repo_type='model', repo_id='srinivasan-sridhar28/Tiny-Storyteller')

In [33]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

context_length = 128
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = ds.map(
    lambda x: tokenizer(x["text"]), batched=True, remove_columns=ds["train"].column_names, batch_size = 2000
)
next(iter(tokenized_datasets["train"]))


Map:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/21990 [00:00<?, ? examples/s]

{'input_ids': [3198,
  1110,
  11,
  257,
  1310,
  2576,
  3706,
  20037,
  1043,
  257,
  17598,
  287,
  607,
  2119,
  13,
  1375,
  2993,
  340,
  373,
  2408,
  284,
  711,
  351,
  340,
  780,
  340,
  373,
  7786,
  13,
  20037,
  2227,
  284,
  2648,
  262,
  17598,
  351,
  607,
  1995,
  11,
  523,
  673,
  714,
  34249,
  257,
  4936,
  319,
  607,
  10147,
  13,
  198,
  198,
  43,
  813,
  1816,
  284,
  607,
  1995,
  290,
  531,
  11,
  366,
  29252,
  11,
  314,
  1043,
  428,
  17598,
  13,
  1680,
  345,
  2648,
  340,
  351,
  502,
  290,
  34249,
  616,
  10147,
  1701,
  2332,
  1995,
  13541,
  290,
  531,
  11,
  366,
  5297,
  11,
  20037,
  11,
  356,
  460,
  2648,
  262,
  17598,
  290,
  4259,
  534,
  10147,
  526,
  198,
  198,
  41631,
  11,
  484,
  4888,
  262,
  17598,
  290,
  384,
  19103,
  262,
  4936,
  319,
  20037,
  338,
  10147,
  13,
  632,
  373,
  407,
  2408,
  329,
  606,
  780,
  484,
  547,
  7373,
  290,
  5742,
  1123,
  584,
  13,
 

In [34]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HqqConfig
import torch
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

hqq_config = HqqConfig(nbits=1, group_size=32, quant_zero=False, quant_scale=False, axis=0) #axis=0 is used by default


model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2", quantization_config=config)
print(model.get_memory_footprint()/ (1024*1024))
model

`low_cpu_mem_usage` was None, now set to True since model is quantized.


101.4858512878418


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [40]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, replace_lora_weights_loftq

peft_config = LoraConfig(target_modules=["c_attn", "c_proj", "c_fc"],task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)


model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 82,502,400 || trainable%: 0.7149


In [41]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = tokenizer.eos_token
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [48]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="Tiny-Storyteller",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    # gradient_accumulation_steps=4,
    num_train_epochs=3,
    optim="adamw_hf",
    max_steps=1000,
    # bf16=False,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)



/home/srinivasan/.local/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [49]:
trainer.train()

/home/srinivasan/.local/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/srinivasan/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 486.00 MiB. GPU 

In [ ]:
trainer.push_to_hub("Tiny-Storyteller")